### Math

In the following, we're going to look at a model for the evolution of the thickness $h$ of the layer of water over a landscape of elevation $z$ subject to a rainfall rate $\dot r$.
Let $u$ be the depth-averaged velocity of the water layer, $g$ the acceleration due to gravity, and $k$ the friction coefficient between the water and the land.
The physical model that we start from is the Saint-Venant equations:

$$\begin{align}
\frac{\partial}{\partial t}h + \nabla\cdot hu & = \dot r \\
\frac{\partial}{\partial t}hu + \nabla\cdot\left(hu\otimes u + \frac{1}{2}gh^2I\right) & = -gh\nabla z - k|u|u
\end{align}$$

where the first equation represents conservation of mass and the second conservation of momentum.
These equations are very difficult to solve when the water layer thickness can go to zero.
To simplify things a bit, we'll make the assumption that the *Froude number* is small, in which case most of the terms on the left-hand side of the momentum equation are zero.
We're then left with just a balance between gravity and friction:

$$k|u|u = -gh\nabla(z + h).$$

By substituting this into the mass transport equation, we're left with a single PDE for the water layer thickness.
The PDE is, admittedly, a little rowdy but I think we'll muddle through.
Before embarking, we should figure out some reasonable values for all of the inputs and coefficients.
The friction coefficient $k$ is dimensionless; [Rousseau et al. (2015)](http://dx.doi.org/10.1061/(ASCE)HE.1943-5584.0001171) use $k = 1 / 32$ in their experiments and we'll follow suit.
During a decent downpour, we might have rainfall values on the order of 10 to 50 mm / hour.
Just to make the units work out super conveniently we'll take $\dot r$ to be 36 mm / hour = 0.036 m / 3600 s = 10${}^{-6}$ m / s.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import firedrake
from firedrake import (
    exp,
    sqrt,
    Constant,
    max_value,
    inner,
    sym,
    grad,
    div,
    dx,
    ds,
    NonlinearVariationalProblem as Problem,
    NonlinearVariationalSolver as Solver,
)

In [ ]:
nx, ny = 64, 64
Lx, Ly = 200.0, 200.0
mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly, diagonal="crossed")
x = firedrake.SpatialCoordinate(mesh)

In [ ]:
Q = firedrake.FunctionSpace(mesh, "CG", 1)
V = firedrake.VectorFunctionSpace(mesh, "CG", 1)

In [ ]:
x_0 = Constant((Lx / 2, Ly / 2))
R = Constant(20.0)
z_0 = Constant(20.0)
expr = z_0 * exp(-inner(x - x_0, x - x_0) / R ** 2)
z = firedrake.interpolate(expr, Q)

In [ ]:
g = Constant(9.8)
k = Constant(1 / 32)
r = Constant(1e-6)

In [ ]:
u = firedrake.Function(V)
h = firedrake.Function(Q)

Every time Daniel says "minimization principle" you take a shot of tequila!

In [ ]:
ε = sym(grad(u))
μ = firedrake.CellSize(mesh)
u_n = u.copy(deepcopy=True)
J = (
    0.5 * (μ + h) * inner(ε, ε)
    #+ k / 3 * inner(u, u) ** (3 / 2)
    + k / 2 * sqrt(inner(u_n, u_n)) * inner(u, u)
    + g * h * inner(grad(z + h), u)
) * dx

F = firedrake.derivative(J, u)
problem_params = {"form_compiler_parameters": {"quadrature_degree": 4}}
problem_u = Problem(F, u, **problem_params)
solver_params = {
    "solver_parameters": {
        #"snes_type": "newtontr",
        "ksp_type": "gmres",
        "pc_type": "jacobi",
        #"pc_factor_mat_solver_type": "mumps",
        "snes_converged_reason": None,
        "snes_monitor": None,
        "snes_view": None,
        "ksp_converged_reason": None,
        "ksp_monitor_true_residual": None,
        "ksp_view": None,
    }
}
solver_u = Solver(problem_u, **solver_params)
solver_u.solve()

In [ ]:
firedrake.norm(u)

In [ ]:
h_n = h.copy(deepcopy=True)
η = firedrake.TestFunction(Q)

ν = firedrake.FacetNormal(mesh)
u_ν = max_value(0, inner(u, ν))

δt = firedrake.Constant(1.)
F = (
    ((h - h_n) * η - δt * (inner(h * u, grad(η)) + r * η)) * dx
    + δt ** 2 * div(h * u) * inner(u, grad(η)) * dx
    + δt * h * u_ν * η * ds
    + δt ** 2 * div(h * u) * u_ν * η * ds
)
problem_h = Problem(F, h)
solver_params = {
    "solver_parameters": {
        "ksp_type": "preonly",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    }
}
solver_h = Solver(problem_h, **solver_params)

In [ ]:
#final_time = 60
#num_steps = int(final_time / float(δt))
num_steps = 1
for t in tqdm.trange(num_steps):
    solver_h.solve()
    h.interpolate(max_value(0, h))
    solver_u.solve()

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(h, axes=axes)
fig.colorbar(colors);

In [ ]:
solver_u.solve()

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(u, axes=axes)
fig.colorbar(colors);